In [45]:
# We will start by constructing the same variables we did in wage_analysis.ipynb
import pandas as pd, numpy as np, matplotlib, matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.stats.weightstats import DescrStatsW
import statsmodels.api as sm
import seaborn as sns
import math

In [2]:
# Loading in the survey data
cis_df_2017 = pd.read_csv('CIS-72M0003-E-2017_F1.csv')
cis_df_2015 = pd.read_csv('CIS-72M0003-E-2015_F1.csv')
scf_df_1982 = pd.read_csv('scf-13M00004-E-1982-ind_F1.csv')
scf_df_1991 = pd.read_csv('scf-13M00004-E-1991-ind_F1.csv')

In [3]:
# Consumer price index
cpi_df = pd.read_csv('cpi_annual.csv')

In [4]:
cpi_df = cpi_df[cpi_df['Products and product groups'] == 'All-items'] # only check the aggregate cpi value
cpi_df = cpi_df[['REF_DATE', 'VALUE']]
# The reference date defaults to 2002, but we would rather have it be 2020
cpi_df['VALUE_2020'] = ( cpi_df['VALUE'] / cpi_df.iloc[-1]['VALUE'] ) * 100

In [5]:
# filter out invalid values
cis_df_2017 = cis_df_2017[(cis_df_2017['WGSAL'] < 99999996) & (cis_df_2017['ALHRWK'] < 9996)]
cis_df_2015 = cis_df_2015[(cis_df_2015['WGSAL'] < 99999996) & (cis_df_2015['ALHRWK'] < 9996)]

# filter out 0 hours
scf_df_1982 = scf_df_1982[(scf_df_1982['HRSWRK'] > 0) & (scf_df_1982['WKSWRK'] > 0)]
scf_df_1991 = scf_df_1991[(scf_df_1991['USHOURS'] > 0) & (scf_df_1991['WKSWRKYR'] > 0)]
cis_df_2017 = cis_df_2017[cis_df_2017['ALHRWK'] > 0]
cis_df_2015 = cis_df_2015[cis_df_2015['ALHRWK'] > 0]

In [6]:
cis_df_2017['HOURLY_WAGE'] = cis_df_2017['WGSAL'] / cis_df_2017['ALHRWK']
cis_df_2015['HOURLY_WAGE'] = cis_df_2015['WGSAL'] / cis_df_2015['ALHRWK']
scf_df_1982['HOURLY_WAGE'] = scf_df_1982['WAGSAL'] / scf_df_1982['HRSWRK'] / scf_df_1982['WKSWRK'] 
scf_df_1991['HOURLY_WAGE'] = scf_df_1991['WAGSAL'] / scf_df_1991['USHOURS'] / scf_df_1991['WKSWRKYR']

In [7]:
scf_df_1982['HOURLY_WAGE_2020'] = scf_df_1982['HOURLY_WAGE'] / (cpi_df.loc[cpi_df['REF_DATE'] == 1982, 'VALUE_2020'].values[0] / 100)
scf_df_1991['HOURLY_WAGE_2020'] = scf_df_1991['HOURLY_WAGE'] / (cpi_df.loc[cpi_df['REF_DATE'] == 1991, 'VALUE_2020'].values[0] / 100)
cis_df_2015['HOURLY_WAGE_2020'] = cis_df_2015['HOURLY_WAGE'] / (cpi_df.loc[cpi_df['REF_DATE'] == 2015, 'VALUE_2020'].values[0] / 100)
cis_df_2017['HOURLY_WAGE_2020'] = cis_df_2017['HOURLY_WAGE'] / (cpi_df.loc[cpi_df['REF_DATE'] == 2017, 'VALUE_2020'].values[0] / 100)

In [8]:
# Average minimum wages every year across all provinces
min_wage_df = pd.read_csv('minwage.csv')
min_wage_df = min_wage_df[min_wage_df['year'] >= 1982] # remove rows not in cpi df
min_wage_df = min_wage_df[min_wage_df['year'] <= 2017]

In [9]:
# Must first convert nominal minwage to real minwage

def to_real_wage(r):
    year = r['year']
    return r['minwage'] / (cpi_df.loc[cpi_df['REF_DATE'] == year, 'VALUE_2020'].values[0] / 100)

min_wage_df['minwage_2020'] = min_wage_df.apply(lambda row : to_real_wage(row), axis=1)

In [10]:
# remove rows with wage below 50% of minimum wage
scf_df_1982 = scf_df_1982[scf_df_1982['HOURLY_WAGE_2020'] > 0.5 * min_wage_df.loc[min_wage_df['year'] == 1982, 'minwage_2020'].values[0] ]
scf_df_1991 = scf_df_1991[scf_df_1991['HOURLY_WAGE_2020'] > 0.5 * min_wage_df.loc[min_wage_df['year'] == 1991, 'minwage_2020'].values[0] ]
cis_df_2015 = cis_df_2015[cis_df_2015['HOURLY_WAGE_2020'] > 0.5 * min_wage_df.loc[min_wage_df['year'] == 2015, 'minwage_2020'].values[0] ]
cis_df_2017 = cis_df_2017[cis_df_2017['HOURLY_WAGE_2020'] > 0.5 * min_wage_df.loc[min_wage_df['year'] == 2017, 'minwage_2020'].values[0] ]

In [11]:
# Only keep individuals between the age of 25 - 60
scf_df_1982 = scf_df_1982[(scf_df_1982['AGE'] <= 60) & (scf_df_1982['AGE'] >= 25)]
scf_df_1991 = scf_df_1991[(scf_df_1991['AGE'] <= 60) & (scf_df_1991['AGE'] >= 25)]
cis_df_2017 = cis_df_2017[(cis_df_2017['AGEGP'] <= 13) & (cis_df_2017['AGEGP'] >= 7)]
cis_df_2015 = cis_df_2015[(cis_df_2015['AGEGP'] <= 13) & (cis_df_2015['AGEGP'] >= 7)]

In [12]:
# Remove those who work less than 260 hours
scf_df_1982['HOURS'] = scf_df_1982['HRSWRK'] * scf_df_1982['WKSWRK'] 
scf_df_1991['HOURS'] = scf_df_1991['USHOURS'] * scf_df_1991['WKSWRKYR']
scf_df_1982 = scf_df_1982[scf_df_1982['HOURS'] >= 260]
scf_df_1991 = scf_df_1991[scf_df_1991['HOURS'] >= 260]
cis_df_2015 = cis_df_2015[cis_df_2015['ALHRWK'] >= 260]
cis_df_2017 = cis_df_2017[cis_df_2017['ALHRWK'] >= 260]

In [13]:
# Create an estimation for potential experience
def cis_schooling_estimate(row):
    group = row['HLEV2G']
    if (group == 1):
        # less than highschool
        return 10
    elif (group == 2):
        # highschool grad
        return 12
    elif (group == 3):
        # non-university postsecondary certificate
        return 13
    elif (group == 4):
        # uni grad
        return 16
    else:
        # invalid
        return 999

In [14]:
def scf_1991_schooling_estimate(row):
    group = row['EDUCREC']
    if (group <= 3):
        # no schooling or grade 8 or lower
        return 10
    elif (group == 4):
        # highschool grad
        return 12
    elif (group <= 6):
        # post-secondary certificate
        return 13
    elif (group == 7):
        # university degree
        return 16
    else:
        return 999

In [15]:
def scf_1982_schooling_estimate(row):
    group = row['EDUC']
    if (group <= 3):
        # no highschool
        return 10
    elif (group <= 5):
        # highschool
        return 12
    elif (group <= 7):
        # post-secondary certificatae
        return 13
    elif (group == 8):
        # university degree
        return 16
    else:
        return 999

In [16]:
scf_df_1982['YEARS_SCHOOLING'] = scf_df_1982.apply(scf_1982_schooling_estimate, axis=1)
scf_df_1991['YEARS_SCHOOLING'] = scf_df_1991.apply(scf_1991_schooling_estimate, axis=1)
cis_df_2015['YEARS_SCHOOLING'] = cis_df_2015.apply(cis_schooling_estimate, axis=1)
cis_df_2017['YEARS_SCHOOLING'] = cis_df_2017.apply(cis_schooling_estimate, axis=1)

In [18]:
scf_df_1982 = scf_df_1982[scf_df_1982['YEARS_SCHOOLING'] != 999]
scf_df_1991 = scf_df_1991[scf_df_1991['YEARS_SCHOOLING'] != 999]
cis_df_2015 = cis_df_2015[cis_df_2015['YEARS_SCHOOLING'] != 999]
cis_df_2017 = cis_df_2017[cis_df_2017['YEARS_SCHOOLING'] != 999]

In [19]:
# Create an estimation for 'potential experience' as age - max(YEARS_SCHOOLING, 10) - 6

def cis_age(row):
    group = int(row['AGEGP'])
    if (group == 5):
        return 19
    elif (group == 6):
        return 22
    elif (group == 7):
        return 27
    elif (group == 8):
        return 32
    elif (group == 9):
        return 37
    elif (group == 10):
        return 42
    elif (group == 11):
        return 47
    elif (group == 12):
        return 52
    elif (group == 13):
        return 57
    elif (group == 14):
        return 62
    elif (group == 15):
        return 67

In [20]:
# Let's do the same for scf so the number of groups is consistent across years
def scf_age(row):
    age = int(row['AGE'])
    if (age < 25): # We can be smart and skip ages outside of 20-60 because we already dropped them
        return 22
    elif (age < 30):
        return 27
    elif (age < 35):
        return 32
    elif (age < 40):
        return 37
    elif (age < 45):
        return 42
    elif (age < 50):
        return 47
    elif (age < 55):
        return 52
    else:
        return 57

In [21]:
for df in [cis_df_2015, cis_df_2017]:
    df['AGE'] = df.apply(cis_age, axis=1)
    
for df in [scf_df_1982, scf_df_1991]:
    df['AGE'] = df.apply(scf_age, axis=1)

In [22]:
for df in [scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]:
    df['POTENTIAL_EXP'] = df['AGE'] - df['YEARS_SCHOOLING'] - 6

In [25]:
cis_df_2017['POTENTIAL_EXP'].describe()

count    31132.000000
mean        22.944719
std         10.400292
min          5.000000
25%         14.000000
50%         23.000000
75%         33.000000
max         41.000000
Name: POTENTIAL_EXP, dtype: float64

In [28]:
# Let's rename the weight columns so we can perform general operations on the dataframes
scf_df_1982 = scf_df_1982.rename(columns={'REVWEIG': 'FWEIGHT'})
scf_df_1991 = scf_df_1991.rename(columns={'WEIGHT': 'FWEIGHT'})

In [40]:
# Now we begin the new stuff. Start by summarizing what we have so far.
summary_df = pd.DataFrame(columns=['mean_wage', 'log_variance_of_wage', 'mean_schooling', 'variance_schooling', 'mean_experience', 'variance_experience'])

In [41]:
summary_df

,mean_wage,log_variance_of_wage,mean_schooling,variance_schooling,mean_experience,variance_experience


In [42]:
years = ['1982', '1991', '2015', '2017']
for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    df['LOG_HOURLY_WAGE_2020'] = df['HOURLY_WAGE_2020'].apply(lambda x : math.log(x))
    
    w_df = DescrStatsW(df['HOURLY_WAGE_2020'], weights=df['FWEIGHT'])
    mean_wage = w_df.mean
    
    l_df = DescrStatsW(df['LOG_HOURLY_WAGE_2020'], weights=df['FWEIGHT'])
    log_var_wage = l_df.var
    
    y_df = DescrStatsW(df['YEARS_SCHOOLING'], weights=df['FWEIGHT'])
    mean_years = y_df.mean
    var_years = y_df.var
    
    e_df = DescrStatsW(df['POTENTIAL_EXP'], weights=df['FWEIGHT'])
    mean_exp = e_df.mean
    var_exp = e_df.var
    
    summary_df.loc[years[i]] = [mean_wage, log_var_wage, mean_years, var_years, mean_exp, var_exp]
summary_df   

,mean_wage,log_variance_of_wage,mean_schooling,variance_schooling,mean_experience,variance_experience
1982,25.920999,0.275562,12.092422,4.199797,20.923845,109.443003
1991,27.024228,0.308400,12.643560,3.577457,20.379538,92.810667
2015,34.241340,0.362288,13.732381,3.605649,21.977420,109.006296
2017,34.225855,0.359003,13.815370,3.618849,21.762212,110.104129


In [43]:
# Although wage inequality seems to increase over time, schooling and experience are becoming less equal
# This suggests that other causes are responsible for the increasing inequality

In [154]:
# Let's run a Mincer regression on log wages

mincer_df = pd.DataFrame(columns=years)

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    df['POTENTIAL_EXP_SQUARED'] = df['POTENTIAL_EXP'].apply(lambda x : x*x)
    # Use a weighted least squares estimate
    basic_result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED']], prepend=False), weights=df['FWEIGHT']).fit()
    params = basic_result.params
    params['R_SQUARED'] = basic_result.rsquared
    params['NUM_OBSERVATIONS'] = len(df)
    mincer_df[years[i]] = params
    
mincer_df.round(5)



,1982,1991,2015,2017
YEARS_SCHOOLING,0.08494,0.08934,0.10148,0.10368
POTENTIAL_EXP,0.02426,0.02752,0.03770,0.04013
POTENTIAL_EXP_SQUARED,-0.00040,-0.00041,-0.00061,-0.00062
const,1.80758,1.66202,1.47884,1.39628
R_SQUARED,0.09333,0.09197,0.11431,0.12680
NUM_OBSERVATIONS,25994.00000,33449.00000,20596.00000,31132.00000


In [63]:
# Let's also calculate the return to experience and schooling (evaluated at mean experience in 2017)

returns_df = pd.DataFrame(columns=years)

for y in years:
    exp_return = mincer_df.loc['POTENTIAL_EXP'][y] + 2 * mincer_df.loc['POTENTIAL_EXP_SQUARED'][y] * summary_df.loc[y]['mean_experience']
    
    sch_return = mincer_df.loc['YEARS_SCHOOLING'][y]
    
    series = pd.Series([exp_return, sch_return])
    series.index = ['Return to Experience', 'Return to Schooling']
    returns_df[y] = series

returns_df
    

,1982,1991,2015,2017
Return to Experience,0.007393,0.010722,0.011088,0.013106
Return to Schooling,0.084943,0.089344,0.101479,0.103683


In [ ]:
# Here we can see that one additional year of experience increases wages by ~ 1.3%
# while one additional year of schooling will increase them by about 10.5% [(e^(0.1) - 1) * 100]

In [76]:
# Let's add in marital status
# First we have to normalize the dummy variables across surveys
scf_df_1991['MARRIED'] = pd.get_dummies(scf_df_1991['MARSTAT'])[2]
scf_df_1982['MARRIED'] = pd.get_dummies(scf_df_1982['MARSTAT'])[2]
cis_df_2015['MARRIED'] = pd.get_dummies(cis_df_2015['MARST'])[1]
cis_df_2017['MARRIED'] = pd.get_dummies(cis_df_2017['MARST'])[1]

In [78]:
mincer_df_2 = pd.DataFrame(columns=years)

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED', 'MARRIED']], prepend=False), weights=df['FWEIGHT']).fit()
    params = result.params
    params['R_SQUARED'] = result.rsquared
    mincer_df_2[years[i]] = params
    
mincer_df_2.round(5)

,1982,1991,2015,2017
YEARS_SCHOOLING,0.08551,0.08974,0.09841,0.09984
POTENTIAL_EXP,0.02169,0.02516,0.03476,0.03622
POTENTIAL_EXP_SQUARED,-0.00036,-0.00037,-0.00056,-0.00056
MARRIED,0.10101,0.08388,0.06794,0.08711
const,1.75008,1.62165,1.52417,1.45515
R_SQUARED,0.09933,0.09602,0.11713,0.13150


In [175]:
# Let's add in sex
mincer_df_3 = pd.DataFrame(columns=years)

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    # Dummies for sex and marital status
    df[['FEMALE_MARRIED', 'MALE_MARRIED', 'FEMALE_SINGLE', 'MALE_SINGLE']] = pd.get_dummies(df.groupby(['SEX','MARRIED'], sort=False).ngroup()).rename(columns={0: 'FEMALE_MARRIED', 1: 'MALE_MARRIED', 2: 'FEMALE_SINGLE', 3: 'MALE_SINGLE'})
    
    # Maintain linear independence of exog by dropping the constant
    result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED', 'FEMALE_MARRIED', 'MALE_MARRIED', 'FEMALE_SINGLE', 'MALE_SINGLE']], weights=df['FWEIGHT']).fit()
    params = result.params
    params['R_SQUARED'] = result.rsquared
    mincer_df_3[years[i]] = params
    
mincer_df_3.round(5)

,1982,1991,2015,2017
YEARS_SCHOOLING,0.08404,0.08685,0.10339,0.10570
POTENTIAL_EXP,0.02184,0.02512,0.03369,0.03502
POTENTIAL_EXP_SQUARED,-0.00038,-0.00039,-0.00054,-0.00053
FEMALE_MARRIED,1.63528,1.56063,1.54746,1.31978
MALE_MARRIED,2.01154,1.73357,1.37019,1.60297
FEMALE_SINGLE,1.85215,1.90552,1.38525,1.30029
MALE_SINGLE,1.69584,1.59536,1.66746,1.45707
R_SQUARED,0.19682,0.17136,0.15592,0.16781


In [172]:
# Let's add in sex as a separate variable
mincer_df_3 = pd.DataFrame(columns=years)

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    # Dummies for sex and marital status
    df[['MALE', 'FEMALE']] = pd.get_dummies(df['SEX']).rename(columns={1: 'MALE', 2: 'FEMALE'})
    
    result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED', 'MARRIED', 'FEMALE']], prepend=False), weights=df['FWEIGHT']).fit()
    params = result.params
    params['R_SQUARED'] = result.rsquared
    mincer_df_3[years[i]] = params
    
mincer_df_3.round(5)

,1982,1991,2015,2017
YEARS_SCHOOLING,0.08491,0.08778,0.10424,0.10639
POTENTIAL_EXP,0.02250,0.02564,0.03362,0.03507
POTENTIAL_EXP_SQUARED,-0.00038,-0.00039,-0.00053,-0.00053
MARRIED,0.05592,0.07366,0.06964,0.08558
FEMALE,-0.32619,-0.29365,-0.23268,-0.22118
const,1.91647,1.78711,1.56078,1.47633
R_SQUARED,0.18966,0.16506,0.15404,0.16504


In [125]:
# Let's test to see if the difference in return to schooling from 1982 and 2017 is significant
# First create a dummy variable
scf_df_1982['2017'] = 0
cis_df_2017['2017'] = 1
# combine the surveys
comb_df = scf_df_1982.append(cis_df_2017)

comb_df['2017 X SCHOOLING'] = comb_df['2017'] * comb_df['YEARS_SCHOOLING']

In [126]:
comb_df['2017 X SCHOOLING']

0         0
4         0
12        0
13        0
22        0
         ..
92274    16
92275    16
92277    16
92278    12
92291    16
Name: 2017 X SCHOOLING, Length: 57126, dtype: int64

In [128]:
# Test the null hypothesis
result = sm.WLS(endog=comb_df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(comb_df[['YEARS_SCHOOLING', '2017', '2017 X SCHOOLING']], prepend=False), weights=comb_df['FWEIGHT']).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             WLS Regression Results                             
================================================================================
Dep. Variable:     LOG_HOURLY_WAGE_2020   R-squared:                       0.102
Model:                              WLS   Adj. R-squared:                  0.102
Method:                   Least Squares   F-statistic:                     2169.
Date:                  Sun, 14 Nov 2021   Prob (F-statistic):               0.00
Time:                          14:38:57   Log-Likelihood:                -59139.
No. Observations:                 57126   AIC:                         1.183e+05
Df Residuals:                     57122   BIC:                         1.183e+05
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
YEARS_SCHOOLING      0.0711      0.002     37.923      0.000       0.067       0.075
2017                -0.0763      0.031     -2.439      0.015      -0.138      -0.015
2017 X SCHOOLING     0.0124      0.002      5.146      0.000       0.008       0.017
const                2.2619      0.023     98.360      0.000       2.217       2.307
==============================================================================
Omnibus:                     7431.530   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            94952.323
Skew:                          -0.015   Prob(JB):                         0.00
Kurtosis:                       9.316   Cond. No.                         265.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [129]:
# The t-value is 5.146 hence we can safely reject the null and the return to schooling significantly increased

In [130]:
# Over time, group composition may have changed
# Let's see how much inequality can be explained in 1982 using population attributes from 2017

In [132]:
# First, find the distribution of yeras of schooling
cis_df_2017['YEARS_SCHOOLING'].value_counts(normalize=True) * 100

13    37.932031
16    33.595657
12    22.414236
10     6.058075
Name: YEARS_SCHOOLING, dtype: float64

In [145]:
schooling2017_df = DescrStatsW(df['YEARS_SCHOOLING'], weights=df['FWEIGHT'])
repeats = schooling2017_df.asrepeats()
unique, counts = np.unique(repeats, return_counts=True)
counts = counts / schooling2017_df.sum_weights
dict_2017 = dict(zip(unique, counts))
dict_2017

{10: 0.05414479392695003,
 12: 0.21510337395626808,
 13: 0.33209303331257356,
 16: 0.39737682165652954}

In [144]:
# Check the distribution in 1982
schooling1982_df = DescrStatsW(scf_df_1982['YEARS_SCHOOLING'], weights=scf_df_1982['FWEIGHT'])
repeats = schooling1982_df.asrepeats()
unique, counts = np.unique(repeats, return_counts=True)
counts = counts / schooling1982_df.sum_weights
dict_1982 = dict(zip(unique, counts))
dict_1982

{10: 0.37794204678559784,
 12: 0.23581791574586175,
 13: 0.23221796593934838,
 16: 0.15402207152919203}

In [148]:
def adjust_weight(row):
    old_weight = row['FWEIGHT']
    sch = row['YEARS_SCHOOLING']
    return old_weight * (dict_2017[sch] / dict_1982[sch])

scf_df_1982['2017_WEIGHT'] = scf_df_1982.apply(adjust_weight, axis=1)

In [149]:
# Adjusted weights
schooling1982_df = DescrStatsW(scf_df_1982['YEARS_SCHOOLING'], weights=scf_df_1982['2017_WEIGHT'])
repeats = schooling1982_df.asrepeats()
unique, counts = np.unique(repeats, return_counts=True)
counts = counts / schooling1982_df.sum_weights
dict_adj_1982 = dict(zip(unique, counts))
dict_adj_1982

{10: 0.05346700290635843,
 12: 0.21495693314154016,
 13: 0.33208280379119265,
 16: 0.3976138803535683}

In [152]:
# There seems to be some small error in the adjustment but overall acceptable

In [153]:
# Now let's estimate coefficients for 1982 using 2017 demographics

In [150]:
result = sm.WLS(endog=scf_df_1982['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(scf_df_1982[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED']], prepend=False), weights=scf_df_1982['2017_WEIGHT']).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             WLS Regression Results                             
================================================================================
Dep. Variable:     LOG_HOURLY_WAGE_2020   R-squared:                       0.120
Model:                              WLS   Adj. R-squared:                  0.120
Method:                   Least Squares   F-statistic:                     1181.
Date:                  Sun, 14 Nov 2021   Prob (F-statistic):               0.00
Time:                          16:22:10   Log-Likelihood:                -30312.
No. Observations:                 25994   AIC:                         6.063e+04
Df Residuals:                     25990   BIC:                         6.066e+04
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
YEARS_SCHOOLING           0.0943      0.002     56.228      0.000       0.091       0.098
POTENTIAL_EXP             0.0347      0.001     24.512      0.000       0.032       0.037
POTENTIAL_EXP_SQUARED    -0.0006   3.33e-05    -19.241      0.000      -0.001      -0.001
const                     1.5905      0.028     56.518      0.000       1.535       1.646
==============================================================================
Omnibus:                     5484.723   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            65123.117
Skew:                          -0.675   Prob(JB):                         0.00
Kurtosis:                      10.636   Cond. No.                     5.22e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [151]:
# R^2 increased from 0.09 to 0.12
# so our estimate of explained inequality increases in 1982 when we use the 2017 schooling composition.

In [162]:
# Let's analyze the residuals of the basic regression

residuals_df = pd.DataFrame(columns=years, index=['Residual Variance'])

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
    
    result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED']], prepend=False), weights=df['FWEIGHT']).fit()
    residuals_df[years[i]] = [result.resid.var()]
    
residuals_df

,1982,1991,2015,2017
Residual Variance,0.262224,0.284891,0.314146,0.300741


In [169]:
# Conclude by addding a self-employment flag
scf_df_1982['SELF_EMPLOYED'] = scf_df_1982['CLASS'].apply(lambda x : 1 if x == 3 else 0)
scf_df_1991['SELF_EMPLOYED'] = scf_df_1991['CLSWRKS'].apply(lambda x : 1 if x == 3 else 0)
cis_df_2015['SELF_EMPLOYED'] = cis_df_2015['FSEMP'].apply(lambda x : 1 if x == 1 else 0)
cis_df_2017['SELF_EMPLOYED'] = cis_df_2017['FSEMP'].apply(lambda x : 1 if x == 1 else 0)

mincer_df_4 = pd.DataFrame(columns=years)

for i, df in enumerate([scf_df_1982, scf_df_1991, cis_df_2015, cis_df_2017]):
        
    result = sm.WLS(endog=df['LOG_HOURLY_WAGE_2020'], exog=sm.add_constant(df[['YEARS_SCHOOLING', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQUARED', 'MARRIED', 'FEMALE', 'SELF_EMPLOYED']], prepend=False), weights=df['FWEIGHT']).fit()
    params = result.params
    params['R_SQUARED'] = result.rsquared
    params['R_SQUARED_ADJ'] = result.rsquared_adj
    mincer_df_4[years[i]] = params
    
mincer_df_4.round(5)


,1982,1991,2015,2017
YEARS_SCHOOLING,0.08460,0.08787,0.10567,0.10862
POTENTIAL_EXP,0.02277,0.02570,0.03424,0.03618
POTENTIAL_EXP_SQUARED,-0.00039,-0.00039,-0.00054,-0.00054
MARRIED,0.05712,0.07224,0.07471,0.09326
FEMALE,-0.32950,-0.29732,-0.24306,-0.23327
SELF_EMPLOYED,-0.45563,-0.46807,-0.29064,-0.29007
const,1.92473,1.79422,1.55302,1.45416
R_SQUARED,0.20050,0.17446,0.16916,0.18180
R_SQUARED_ADJ,0.20031,0.17431,0.16892,0.18164


In [170]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             WLS Regression Results                             
================================================================================
Dep. Variable:     LOG_HOURLY_WAGE_2020   R-squared:                       0.182
Model:                              WLS   Adj. R-squared:                  0.182
Method:                   Least Squares   F-statistic:                     1153.
Date:                  Sun, 14 Nov 2021   Prob (F-statistic):               0.00
Time:                          22:03:28   Log-Likelihood:                -32555.
No. Observations:                 31132   AIC:                         6.512e+04
Df Residuals:                     31125   BIC:                         6.518e+04
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
YEARS_SCHOOLING           0.1086      0.002     62.845      0.000       0.105       0.112
POTENTIAL_EXP             0.0362      0.001     25.286      0.000       0.033       0.039
POTENTIAL_EXP_SQUARED    -0.0005   3.14e-05    -17.301      0.000      -0.001      -0.000
MARRIED                   0.0933      0.007     14.301      0.000       0.080       0.106
FEMALE                   -0.2333      0.006    -37.560      0.000      -0.245      -0.221
SELF_EMPLOYED            -0.2901      0.011    -25.247      0.000      -0.313      -0.268
const                     1.4542      0.029     50.358      0.000       1.398       1.511
==============================================================================
Omnibus:                     4253.113   Durbin-Watson:                   1.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            59139.774
Skew:                           0.044   Prob(JB):                         0.00
Kurtosis:                       9.752   Cond. No.                     7.08e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""